# Documentacion

Esta es la documentacion cutre del programa. Espero que me sirva en
algun momento, pero dudo que sea vista por mis revisores, la
verdad...

## Indice

- [Input de usuario](#Input-de-usuario)
- [Espacio de trabajo](#Espacio-de-trabajo)
- [Definicion de elementos](#Definicion-de-elementos)
- [Areas tributarias](#Areas-tributarias)
- [Cargas nodales](#Cargas-nodales)
- [Cargas distribuidas](#Cargas-distribuidas)
- [Analisis estatico](#Analisis-estatico)
- [Combinaciones de carga](#Combinaciones-de-carga)
- [Resultados: esfuerzos nodales](#Resultados:-esfuerzos-nodales)
- [Resultados: Resumen](#Resultados:-Resumen)
- [Resultados: Diagramas](#Resultados:-Diagramas)
- [Resultados: Graficos](#Resultados:-Graficos)
- [Esquema de la estructura](#Esquema-de-la-estructura)
- [Flujo de la aplicacion](#Flujo-de-la-aplicacion)

In [324]:
if MERYTEST := 0:
    import tracemalloc
    tracemalloc.start()
from math import pi as π, sin
from numpy import array, zeros, round as mround
from numpy.linalg import inv, solve
import matplotlib.pyplot as pete

def line(): print('---- -- - -- ----')

def varsC(dirl=dir()):
    i = 0
    lista = []
    ipyvars = ['In', 'Out', 'exit', 'get_ipython', 'quit']
    for j in dirl:
        if j[0] == '_': continue
        elif j in ipyvars: continue
        i += 1
        lista.append((i, j))
    return lista
def period():
    """formulas del paper en base al ejemplo de la tesis"""
    Sc = 67500*8*0.01**4
    Sb = 160000*6*0.01**4
    B = 1.5*3
    H = 1.2*2
    W = 0 # FALTA
    T = 0.374*H**1.1*W/(B**0.3*Sc**0.31*Sb**0.35)
    print(0.09*H/3**0.5)
    print(0.075*H**0.75)
    print(0.0466*H**0.9)
    print(T)
    print(0.879*H**0.43*3**0.65*W**0.18/B)
    print(0.475*H**0.547*3**0.103/B**0.271)
    print(0.428*H**0.545/B**0.185)
    print(0.28*H**0.54)
    print(0.012*H)
    return
p = print

## Input de usuario

- [volver](#Indice)

El usuario final es el encargado de llenar un documento de texto el
cual cuenta con la informacion principal para definir y generar una
estructura de marcos de hormigon armado. La estructura en cuestion,
para esta version del programa, se espera que conserve una simetria a
lo largo de sus ejes de manera independiente, en otras palabras, que
para ambas direcciones de analisis, todas las vigas deben ser iguales
entre si y lo mismo aplica para las columnas.

El input esperado por el usuario contempla la siguiente forma como
ejemplo:

```
numero de pisos          : 2
numero de vanos          : 3
numero de marcos         : 4
altura de piso           : 150 # [cm]
distancia entre columnas : 120 # [cm]
distancia entre marcos   : 120 # [cm]
espesor de losas         : 15 # [cm]

modulo de poisson 𝜈      : 0.25
resistencia f'c hormigon : 19.5 # [MPa]
densidad del hormigon    : 2500 # [kg/m3]
sobrecarga de uso        : 500 # [kg/m2]

zona sismica             : 3
tipo de suelo            : E
categoria de edificacion : II
factor de modificacion R : 5.5
porcentaje de sobrecarga : 0.25 # revisar en la norma el nombre

resistencia fy acero     : 4200 # [kg/cm2]
ancho max de columnas    : 90 # [cm]
ancho max de vigas       : 40 # [cm]
altura max de vigas      : 90 # [cm]
mismas barras en columnas: si
```

La recuperacion de los datos del archivo ingresado por el usuario se
presenta en el siguiente bloque:

In [325]:
# lectura de datos

def lectura():
    """Lee los parametros definidos por el usuario desde un txt. Si\
    se omite el valor, se autocompleta con los valores por defecto."""

    with open('../IO/input.txt', encoding='utf-8') as in_text:
        in_temp = in_text.readlines()

    default = {
        'numero de pisos': '2',
        'numero de vanos': '3',
        'numero de marcos': '4',
        'altura de piso': '240',
        'distancia entre columnas': '300',
        'distancia entre marcos': '300',
        'espesor de losas': '15',

        'modulo de poisson 𝜈': '0.25',
        'resistencia f\'c hormigon': '19.5',
        'densidad del hormigon': '2500',
        'sobrecarga de uso': '500',

        'zona sismica': '3',
        'tipo de suelo': 'E',
        'categoria de edificacion': 'II',
        'factor de modificacion R': '7',
        'porcentaje de sobrecarga': '0.25',

        'ancho de columnas': '90',
        'ancho de vigas': '40',
        'altura de vigas': '90',
    }

    user_op = [] # option names from txt file
    user_in = [] # values given by the user

    for i in in_temp:
        if i.startswith('\n') == False:
            user_op.append(i.partition(':')[0].strip())
            user_in.append(i.partition(':')[2].strip())
    print(user_in)
    del in_temp
    if user_op != list(default.keys()): raise Exception('wena')

    for i in range(len(user_in)):
        if user_in[i] == '':
            user_in[i] = list(default.values())[i]
        elif user_in[i].isalpha() == False:
            user_in[i] = float(user_in[i])
    for i in range(7): user_in[i] = int(user_in[i])
    print(user_in)
    print(user_op)
    return

lectura()

['4', '3', '4', '300', '700', '700', '15', '0.25', '25', '2500', '500', '3', 'E', 'II', '7', '0.25', '90', '40', '90']
[4, 3, 4, 300, 700, 700, 15, 0.25, 25.0, 2500.0, 500.0, 3.0, 'E', 'II', 7.0, 0.25, 90.0, 40.0, 90.0]
['numero de pisos', 'numero de vanos', 'numero de marcos', 'altura de piso', 'distancia entre columnas', 'distancia entre marcos', 'espesor de losas', 'modulo de poisson 𝜈', "resistencia f'c hormigon", 'densidad del hormigon', 'sobrecarga de uso', 'zona sismica', 'tipo de suelo', 'categoria de edificacion', 'factor de modificacion R', 'porcentaje de sobrecarga', 'ancho de columnas', 'ancho de vigas', 'altura de vigas']


## Espacio de trabajo

- [volver](#Indice)

In [326]:
class Grilla():
    """Base para definir las dimensiones una estructura."""
    def __init__(grid, grilla):
        grid.grilla = grilla
        grid.dimXY = grid.dimensions()
        grid.dim = {'x': grid.dimXY[1], 'y': grid.dimXY[0]}
        grid.nodos = grid.nodes()
        grid.max_barras = (grid.dim['x'] - 1)*grid.dim['y'] \
                        + grid.dim['y']*grid.dim['x']
        grid.K = zeros((3*grid.nodos, 3*grid.nodos)) # rigidez del sistema
        grid.r = 0 # desplazamientos
        grid.drift = {}
        grid.annainfo = {}
        grid.VERB = True # true = verbose on
        return
    def clean(grid):
        grid.annainfo = {
            'esfuerzos': {
                'head': [
                    ['perfil', 'combo', 'nodo', 'axial', 'corte', 'momento'],
                    ['[kg]', '[kg]', '[kg-m]']
                ],
                'body': []
            },
            'delta-nodos': {
                'head': [
                    ['combo', 'nodo', 'X', 'Y', 'Giro'],
                    ['[mm]', '[mm]', '[rad]']
                ],
                'body': []
            },
            'drifts': {
                'head': [
                    ['combo', 'nivel', 'δmax', 'δmin', 'drift'],
                    ['[mm]', '[mm]', '[‰]']
                ],
                'body': []
            }
        }
        return
    def dimensions(grid):
        """Retorna la tupla (nodos en Y, nodos en X)."""
        return len(grid.grilla), len(grid.grilla[0])
    def nodes(grid):
        """Retorna la cantidad de nodos."""
        return grid.dimXY[0]*grid.dimXY[1]
    def maxElemets(grid):
        return
    def dibujar(grid):
        marco2, ejes2 = pete.subplots(1, 1, figsize=(8.0,4.0))
        ejes2.set_xticks([i*b_losas for i in range(n_vanos + 1)])
        ejes2.set_yticks([i*h_pisos for i in range(n_pisos + 1)])
        for m in miembro:
            if not m: continue
            ejes2.plot([m.x0, m.x1], [m.y0, m.y1], 'k', linewidth=2)
        marco3 = pete.figure()
        ejes3 = pete.axes(projection='3d')
        ejes3.grid(False)
        ejes3.set_xticks([i*b_losas/100 for i in range(n_vanos + 1)])
        ejes3.set_yticks([i*d_losas/100 for i in range(n_marcos)])
        ejes3.set_zticks([])
        
        z = 0
        for marco in range(1, n_marcos + 1):
            for m in miembro:
                if not m: continue
                if m.name == 'COL' and marco < n_marcos:
                    ejes3.plot3D([m.x1/100, m.x1/100], [z/100, (z + d_losas)/100], [m.y1/100, m.y1/100], 'r', linewidth=0.5)
                if marco == n_marcos - 1:
                    ejes3.plot3D([m.x0/100, m.x1/100], [z/100, z/100], [m.y0/100, m.y1/100], 'k', linewidth=2)
                else:
                    ejes3.plot3D([m.x0/100, m.x1/100], [z/100, z/100], [m.y0/100, m.y1/100], 'b', linewidth=0.5)
            z += d_losas
        
        ejes3.view_init(3, -45)
        marco3.savefig(f'_marco_vista_a.svg')
        ejes3.view_init(6, -45)
        marco3.savefig(f'_marco_vista_b.svg')
        ejes3.view_init(12, -12)
        marco3.savefig(f'_marco_vista_c.svg')
        #ejes3.view_init(60, -72)
        #marco3.savefig(f'_marco_vista_d.svg')
        return

TEST = 3
if TEST == 3:
    struct = Grilla([
            [1,2,3,4],
            [5,6,7,8],
            [9,10,11,12],
            [13,14,15,16],
    ])
elif TEST == 2:
    struct = Grilla([
            [1,2,3,4],
            [5,6,7,8],
    ])
elif TEST == 1:
    struct = Grilla([[1,2]])

vars(struct)

{'grilla': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]],
 'dimXY': (4, 4),
 'dim': {'x': 4, 'y': 4},
 'nodos': 16,
 'max_barras': 28,
 'K': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'r': 0,
 'drift': {},
 'annainfo': {},
 'VERB': True}

In [327]:
# test - Input desde archivo txt
zona = '3'
categoria = 'II'
suelo = 'E'
R = 7


if TEST == 3:
    h_pisos = 300 # [cm] pero despues sera en cm y despues en varias unidades
    n_pisos = 4
    n_vanos = 3
elif TEST == 1:
    h_pisos = 240 # [cm] pero despues sera en cm y despues en varias unidades
    n_pisos = 1
    n_vanos = 1

n_marcos = 4

if TEST == 3:
    d_losas = 700 # [cm]
    b_losas = 700 # [cm]
if TEST == 1:
    d_losas = 300 # [cm]
    b_losas = 300 # [cm]

t_losas = 15 # [cm]
Q_L = 500/100**2 # [kg/cm2]

b_VIG = 40 # [cm]
h_VIG = 90 # [cm]
b_COL = 90 # [cm]
h_COL = 90 # [cm]
ϰ = 0 # 1.2
𝜈 = 0.25
𝛾 = 2500 # [kg/m3]
fc = 25 # [MPa]

## Definicion de elementos

- [volver](#Indice)

En esta seccion, primero se define una clase con todos los atributos
de un elemento 'barra' para luego inicializar tantos elementos como
sean necesarios y se almacenan en una lista.

In [328]:
# definicion de elementos

class barra():
    # agregar rigidos
    def __init__(bar, nodos, i, j, x0, y0, x1, y1, b, h, ϰ, 𝜈, 𝛾, fc):
        """Define los parametros del perfil y material de un elemento 'barra'."""
        
        #### ## input ## ####
        bar.i = i # nodo A
        bar.j = j # nodo B
        bar.x0 = x0 # [cm]
        bar.y0 = y0 # [cm]
        bar.x1 = x1 # [cm]
        bar.y1 = y1 # [cm]
        bar.b = b # [cm]
        bar.h = h # [cm]
        bar.ϰ = ϰ # factor de forma de la seccion
        bar.𝜈 = 𝜈 # modulo de poisson
        bar.𝛾 = 𝛾/100**3 # [kg/m3] -> [kg/cm3] - densidad
        bar.fc = fc # [MPa] - resistencia en compresion
        
        if x0 != x1:
            bar.name = 'VIGA'
        else:
            bar.name = 'COL'

        #### ## internals ## ####
        bar.GdL(i, j, nodos) # grados de libertad

        bar.L = ((x1 - x0)**2 + (y1 - y0)**2)**0.5 # [cm]
        bar.s = (y1 - y0)/bar.L # senos
        bar.c = (x1 - x0)/bar.L # coseno
        bar.A = b*h # [cm2]
        bar.I = b*h**3/12 # [cm4]
        bar.E = 𝛾**1.5*0.043*fc**0.5/9.80665/0.01 # [kg/mm2]/0.01 = [kg/cm2]
        bar.G = bar.E/(2*(1+𝜈)) # modulo de cizalle
        bar.β = 6*bar.E*bar.I*ϰ/(bar.G*bar.A*bar.L**2)

        bar.rigidez_local(bar.L, bar.A, bar.I, bar.E, bar.β)
        bar.compatibilidad_geometrica(bar.s, bar.c, bar.L)

        bar.K = bar.a.T@bar.k@bar.a # rigidez global
        
        bar.dst_loads = [] # (a, b, [kg/cm]) - cargas distribuidas
        bar.jnt_loads = [] # [kg] [kg-cm] - cargas nodales
        bar.caso = {} # suma de todas las cargas basicas por cada caso
        bar.combo = {} # vectores de cada combinacion de casos
        bar.q_caso = {}
        bar.q_combo = {}
        return
            
    def GdL(bar, a, b, nodosEstruc, tipo=''):
        """Prepara la matriz de transformacion de cada barra."""
        bar.T = zeros([6, 3*nodosEstruc])
        if a > nodosEstruc or b > nodosEstruc:
            print('index error')
            return
        if a < 0 or b < 0:
            print('index error')
            return
        if a != 0:
            bar.T[0, 3*a - 3] = 1
            bar.T[1, 3*a - 2] = 1
            bar.T[2, 3*a - 1] = 1
        if b != 0:
            bar.T[3, 3*b - 3] = 1
            bar.T[4, 3*b - 2] = 1
            bar.T[5, 3*b - 1] = 1
        return

    def compatibilidad_geometrica(bar, s, c, L):
        bar.a = array([
            [-s/L, c/L, 1,  s/L, -c/L, 0],
            [-s/L, c/L, 0,  s/L, -c/L, 1],
            [ c,   s,   0, -c,   -s,   0]
        ])
        return

    def rigidez_local(bar, L, A, I, E, β):
        k11 = 2*E*I*(2 + β)/(L*(1 + 2*β))
        k12 = 2*E*I*(1 - β)/(L*(1 + 2*β))
        k21 = k12
        k22 = k11
        k33 = A*E/L
        bar.k = array([
            [k11, k12, 0  ],
            [k21, k22, 0  ],
            [0,   0,   k33]
        ])
        return

    def addDistF(bar, a, b, fa, fb, coord='Y'):
        """Ingresa una fuerza distribuida desde 'a' hasta 'b'."""
        pass

    def sigma(bar, r_global): # sin uso - borrar
        # σ = k@ε
        # ε = a@r_i
        # r_i = T@r
        # σ = k@a@T@r
        bar.σ = bar.k@bar.a@bar.T@r_global # esfuerzos
        
    def dist2joint(bar): pass

    def calcular_peso_propio(bar):
        """Devuelve el peso propio de la barra en kg/cm. La tupla\
        contiene tramo inicial y final donde la carga no es constante."""
        𝛾 = bar.𝛾
        b = bar.b
        h = bar.h
        bar.dst_loads.append(('_pp-bar', 'D', (0, 0, 𝛾*b*h)))
        return
    
    def case_gen(bar, accion='generar'):
        """Genera las categorias de cargas basicas de la barra."""
        if accion == 'generar':
            case_keys = ['D', 'L']
            bar.q_caso = {} # rect dist loads
            for i in case_keys: bar.q_caso.update({i: 0})
            bar.caso = {} # joint loads
            for i in case_keys: bar.caso.update({i: zeros(6)})
        return
    
    def update(bar):
        bar.I = bar.b*bar.h**3/12
        bar.A = bar.b*bar.h
        bar.rigidez_local(bar.L, bar.A, bar.I, bar.E, bar.β)
        bar.K = bar.a.T@bar.k@bar.a # rigidez global
        return
    
    def calculate_el_piso(): pass


    r = 0 # desplazamientos
    ε = 0 # deformaciones
    σ = 0 # esfuerzos en coordenadas locales [Ma, Mb, F]
    sigma = 0 # esfuerzos en coordenadas globales [Na, Va, Ma, Nb, Vb, Mb]
    v = 0 # esfuerzo de corte
    hayPP = False # indexador caso PesoPropio - actualizar en el futuro
    losa = {'D': [], 'L': []} # indexador casos losa
    caso: {} # casos de cargas basicas
    
    
    # despues agregar tramos rigidos

In [329]:
# definicion geometrica de cada elemento

def armar_estructura(test, NODOS):
    """Genera todos los elementos 'barra' de la estructura."""

    barras = []
    if test == 3:
        m1 = barra(NODOS, 0, 1, 0, 0, 0, 300, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m2 = barra(NODOS, 0, 2, 700, 0, 700, 300, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m3 = barra(NODOS, 0, 3, 1400, 0, 1400, 300, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m4 = barra(NODOS, 0, 4, 2100, 0, 2100, 300, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m5 = barra(NODOS, 1, 5, 0, 300, 0, 600, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m6 = barra(NODOS, 2, 6, 700, 300, 700, 600, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m7 = barra(NODOS, 3, 7, 1400, 300, 1400, 600, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m8 = barra(NODOS, 4, 8, 2100, 300, 2100, 600, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m9 = barra(NODOS, 5, 9, 0, 600, 0, 900, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m10 = barra(NODOS, 6, 10, 700, 600, 700, 900, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m11 = barra(NODOS, 7, 11, 1400, 600, 1400, 900, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m12 = barra(NODOS, 8, 12, 2100, 600, 2100, 900, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m13 = barra(NODOS, 9, 13, 0, 900, 0, 1200, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m14 = barra(NODOS, 10, 14, 700, 900, 700, 1200, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m15 = barra(NODOS, 11, 15, 1400, 900, 1400, 1200, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m16 = barra(NODOS, 12, 16, 2100, 900, 2100, 1200, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m17 = barra(NODOS, 1, 2, 0, 300, 700, 300, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m18 = barra(NODOS, 2, 3, 700, 300, 1400, 300, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m19 = barra(NODOS, 3, 4, 1400, 300, 2100, 300, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m20 = barra(NODOS, 5, 6, 0, 600, 700, 600, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m21 = barra(NODOS, 6, 7, 700, 600, 1400, 600, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m22 = barra(NODOS, 7, 8, 1400, 600, 2100, 600, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m23 = barra(NODOS, 9, 10, 0, 900, 700, 900, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m24 = barra(NODOS, 10, 11, 700, 900, 1400, 900, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m25 = barra(NODOS, 11, 12, 1400, 900, 2100, 900, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m26 = barra(NODOS, 13, 14, 0, 1200, 700, 1200, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m27 = barra(NODOS, 14, 15, 700, 1200, 1400, 1200, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m28 = barra(NODOS, 15, 16, 1400, 1200, 2100, 1200, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
    elif test == 2: # tesis
        m1 = barra(NODOS, 0, 1, 0, 0, 0, 120, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m2 = barra(NODOS, 0, 2, 150, 0, 150, 120, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m3 = barra(NODOS, 0, 3, 300, 0, 300, 120, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m4 = barra(NODOS, 0, 4, 450, 0, 450, 120, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m5 = barra(NODOS, 1, 5, 0, 120, 0, 240, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m6 = barra(NODOS, 2, 6, 150, 120, 150, 240, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m7 = barra(NODOS, 3, 7, 300, 120, 300, 240, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m8 = barra(NODOS, 4, 8, 450, 120, 450, 240, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m9  = barra(NODOS, 1, 2, 0, 120, 150, 120, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m10 = barra(NODOS, 2, 3, 150, 120, 300, 120, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m11 = barra(NODOS, 3, 4, 300, 120, 450, 120, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m12 = barra(NODOS, 5, 6, 0, 240, 150, 240, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m13 = barra(NODOS, 6, 7, 150, 240, 300, 240, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
        m14 = barra(NODOS, 7, 8, 300, 240, 450, 240, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)
    elif test == 1: # testing
        m1 = barra(NODOS, 0, 1, 0, 0, 0, 100, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m2 = barra(NODOS, 0, 2, 300, 0, 300, 100, b_COL, h_COL, ϰ, 𝜈, 𝛾, fc)
        m3 = barra(NODOS, 1, 2, 0, 100, 300, 100, b_VIG, h_VIG, ϰ, 𝜈, 𝛾, fc)

    for i in range(1, struct.max_barras + 1):
        exec('barras.append(m' + str(i) + ')')
    return barras

miembro = [None] + armar_estructura(TEST, struct.nodos)

print(miembro[1].a)

[[-0.00333333  0.          1.          0.00333333 -0.          0.        ]
 [-0.00333333  0.          0.          0.00333333 -0.          1.        ]
 [ 0.          1.          0.         -0.         -1.          0.        ]]


#### hack

[factor](#factor)

In [330]:
if 0:
    perfiles_i = [
        (58, 58),(58, 58),(58, 58),(58, 58),(54, 54),(54, 54),(54, 54),(54, 54),(49, 49),(49, 49),(49, 49),(49, 49),(49, 49),(49, 49),(49, 49),(49, 49),(29, 72),(29, 72),(29, 72),(39, 77),(39, 77),(39, 77),(29, 63),(29, 63),(29, 63),(29, 63),(29, 63),(29, 63),
    ]
    for i, m in enumerate(miembro):
        if not m: continue
        (m.b, m.h) = perfiles_i[i - 1]
        m.update()

In [331]:
# K del sistema
for m in miembro:
    if m == None: continue
    struct.K += m.T.T@m.K@m.T

## Areas tributarias

- [volver](#Indice)

El area de una losa que tributa sobre una viga se calcula en funcion
de las condiciones de borde de la misma losa. Se puede generalizar la
forma de cada area por medio de un trapecio escaleno cuyas longitudes
de sus lados proyectadas sobre su base pueden producir que, para la
base siendo igual a la longitud L de la viga en la que se descarga y
las proyecciones de los lados sobre la base denotandolas como a y b,
al momento de descargar la losa se produzca un area triangular si a y
b sumadas dan L.

Los angulos que tendran los lados del trapecio respecto de su base se
establecen con los siguientes criterios en funcion del par de lados
adyacentes de la losa segun corresponda:

- Para una disposicion de lados empotrado y simplemente apoyado: se
traza un rayo entre ambos lados a 60 grados del borde empotrado.
- Para una disposicion de lados de igual condicion de borde: se traza
un rayo equivalente a la bisectriz entre ambos lados (45 grados).

In [332]:
def h_trib_losa(L, d, c_borde, neg=False): #actualizar descripcion
    """Retorna la proyeccion de los lados de un trapecio sobre su\
    base y el valor del tramo constante de la carga distribuida 'q'.
    L       = Largo de la base de la losa planta (viga del marco en\
    analisis)
    d       = Distancia entre marcos
    t       = Espesor de la losa
    𝛾       = Densidad del hormigon
    c_borde = Condicion de borde
    neg     = Indica si las proyecciones 'a' y 'b' se deben voltear
    Por defecto se calcula siempre 'a' adyacente a un angulo de 45\
    grados cuando es posible."""
    if c_borde == 1: pass
    elif c_borde == '2':
        if d <= L:
            h = d/2**0.5*sin(60/180*π)/sin(75/180*π)
        else:
            h = L/2**0.5*sin(60/180*π)/sin(75/180*π)
        a = h
        b = a*2**0.5*sin(75/180*π)/sin(60/180*π) - a
    elif c_borde == '3c':
        k = 2/2**0.5*sin(60/180*π)/sin(75/180*π)
        if d <= k*L:
            h = d/2
        else:
            h = k*L/2
        a = h
        b = a*2**0.5*sin(75/180*π)/sin(60/180*π) - a
    elif c_borde == '3u':
        k = 2**0.5/2*sin(75/180*π)/sin(60/180*π)
        if d <= k*L:
            h = d/2**0.5*sin(60/180*π)/sin(75/180*π)
        else:
            h = L/2
        a = b = h
    elif c_borde == '4' or c_borde == '0':
        print('estoy aca')
        if d <= L:
            h = d/2
        else:
            h = L/2
        a = b = h
    if neg: a, b = b, a
    return [a, b, h]

if TEST: print(h_trib_losa(451.1, 355.77, '3c', 1))

[102.70195263479661, 177.885, 177.885]


## Cargas nodales

- [volver](#Indice)

A partir de un analisis de una viga doblemente empotrada, se calculan
las reacciones de esta para cada caso, considerando la continuidad de
una losa que descargue sobre el marco. Para esto se consideran las
formulas de empotramiento perfecto de cargas lineales y continuas. De
este modo, se obtiene la carga trapezoidal resultante de la
superposicion de tres casos de cargas por viga.

Formula para una carga constante:

In [333]:
def EP_rec(L, q, a, b, d):
    """Calculo de las reacciones en empotramiento perfecto para una carga constante.
    L = Largo del elemento
    q = Magnitud de la carga distribuida
    a = Distancia desde el inicio del elemento hasta la mitad de la aplicacion de la carga
    b = Distancia desde el final del elemento hasta la mitad de la aplicacion de la carga
    d = Largo de la aplicacion de la carga"""
    Ra = q*d/L**3*((2*a + L)*b**2 + (a - b)/4*d**2)
    Rb = q*d/L**3*((2*b + L)*a**2 - (a - b)/4*d**2)
    Ma = q*d/L**2*(a*b**2 + d**2/12*(L - 3*b))
    Mb = -q*d/L**2*(a**2*b + d**2/12*(L - 3*a))
    if struct.VERB: print('Ra Ma Rb Mb', round(Ra, 1), round(Ma, 1), round(Rb, 1), round(Mb, 1))
    return Ra, Ma, Rb, Mb

EP_rec(2, 1000, 0.5 + 0.6, 0.9, 1.2);

Ra Ma Rb Mb 521.1 242.1 678.9 -279.9


Formula para una carga lineal:

In [334]:
def EP_tri(L, q, a):
    """Calculo de las reacciones en empotramiento perfecto para una carga lineal.
    L = Largo del elemento
    q = Magnitud final de la carga distribuida lineal
    a = Distancia desde el inicio del elemento hasta el final de la aplicacion de la carga"""
    Ra = q*a/20*(10 - a**2/L**2*(15 - 8*a/L))
    Rb = q*a**3/(20*L**2)*(15-8*a/L)
    Ma = q*a**2/30*(10 - a/L*(15 - 6*a/L))
    Mb = -q*a**3/(20*L)*(5 - 4*a/L)
    if struct.VERB: print('Ra Ma Rb Mb', round(Ra, 1), round(Ma, 1), round(Rb, 1), round(Mb, 1))
    return Ra, Ma, Rb, Mb

EP_tri(2, 1000, 0.5);

Ra Ma Rb Mb 229.7 55.2 20.3 -12.5


La superposicion de las cargas lineales y constantes se lleva a cabo
en el siguiente bloque, obteniendo las cargas nodales de cada
elemento:

In [335]:
# superposicion de casos de carga de una viga cualquiera respecto del eje Y

def emp_perf_Y(L, a, b, q):
    """Calcula las reacciones de empotramiento perfecto de una viga dada sometida a una carga trapezoidal.
    L = Largo del elemento
    a = Distancia desde el inicio del elemento hasta el inicio de la carga constante
    b = Distancia desde el final del elemento hasta el final de la carga costante
    q = Magnitud de la carga distribuida trapezoidal"""
    r = [[],[],[]]
    joints = []
    d = L - a - b

    *r[0], = EP_tri(L, q, a)
    *r[1], = EP_rec(L, q, a + d/2, b + d/2, d)
    *r[2], = EP_tri(L, q, b)

    r[2][0], r[2][2] = r[2][2], r[2][0]
    r[2][1], r[2][3] = -r[2][3], -r[2][1]

    for i in range(4):
        joints.append(0)
        for reaction in r:
            joints[i] += reaction[i]

    if struct.VERB: # borrame
        line()
        print(
            'Va:', round(joints[0], 2),
            '\nMa:', round(joints[1], 2),
            '\nVb:', round(joints[2], 2),
            '\nMb:', round(joints[3], 2)
        )
    return (0, *joints[0:2], 0, *joints[2:4])

emp_perf_Y(3, 0.75, 1.4, 300)

Ra Ma Rb Mb 103.4 37.3 9.1 -8.4
Ra Ma Rb Mb 167.2 106.7 87.8 -70.5
Ra Ma Rb Mb 158.5 84.4 51.5 -43.0
---- -- - -- ----
Va: 322.07 
Ma: 186.91 
Vb: 255.43 
Mb: -163.34


(0,
 322.0658402777778,
 186.91490625,
 0,
 255.43415972222226,
 -163.34238541666667)

## Cargas distribuidas

- [volver](#Indice)

Cada elemento 'barra' guarda su propia informacion sobre cada fuerza
distribuida que actue directamente sobre este y sus correspondientes
descomposiciones en cargas puntuales en coordenadas globales producto
del calculo de empotramientos perfectos.

In [336]:
# cargas muertas - peso propio

def pp_barras(accion='cargar'):
    """Aplica o limpia las cargas distribuidas y reacciones en los nodos producto del peso propio de todas las barras."""
    if accion == 'limpiar':
        for m in miembro:
            if not m: continue
            if not m.hayPP:
                print('no habia nada que eliminar!')
            else:
                for i, basicLoad in enumerate(m.dst_loads):
                    if basicLoad[0] == '_pp-bar':
                        m.dst_loads.pop(i)
                        m.jnt_loads.pop(i)
                        break
                print('peso propio eliminado.')
                m.hayPP = False
    elif accion == 'cargar':
        for m in miembro:
            if not m: continue
            if m.hayPP: # actualizar PP para borrar el for
                print('ya estaba definido!')
            else:
                m.calcular_peso_propio()
                if m.name == 'VIGA':
                    for i, basicLoad in enumerate(m.dst_loads):
                        if basicLoad[0] == '_pp-bar':
                            m.jnt_loads.append((
                                '_pp-bar',
                                'D',
                                array(emp_perf_Y(
                                    m.L, *m.dst_loads[i][2]
                                ))
                            ))
                            break
                elif m.name == 'COL':
                    Na = m.dst_loads[0][2][2]*m.L/2
                    m.jnt_loads.append(
                        ('_pp-bar', 'D', array([0, Na, 0, 0, Na, 0]))
                    )
                print('peso propio calculado.')
                m.hayPP = True
    return

pp_barras('cargar')
#print(miembro[1].dst_loads)
#print(miembro[2].dst_loads)
#print(miembro[3].dst_loads)
#miembro[2].jnt_loads[0][2]@miembro[2].T

peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
peso propio calculado.
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
Ra Ma Rb Mb 3150.0 367500.0 3150.0 -367500.0
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
---- -- - -- ----
Va: 3150.0 
Ma: 367500.0 
Vb: 3150.0 
Mb: -367500.0
peso propio calculado.
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
Ra Ma Rb Mb 3150.0 367500.0 3150.0 -367500.0
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
---- -- - -- ----
Va: 3150.0 
Ma: 367500.0 
Vb: 3150.0 
Mb: -367500.0
peso propio calculado.
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
Ra Ma Rb Mb 3150.0 367500.0 3150.0 -367500.0
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
---- -- - -- ----
Va: 3150.0 
Ma: 367500.0 
Vb: 3150.0 
Mb: -367500.0
peso propio calculado.
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
Ra Ma Rb Mb 315

Para cada elemento donde descargue una losa se calculan los casos de
carga distribuida viva y muerta segun el area tributarea.
Adicionalmente, se rectifican las cargas trapezoidales para tratarlas
como cargas distribuidas constantes a lo largo de todo el elemento.

In [337]:
# cargas muertas y vivas - descarga losa

def q_losa(m, forma='', caso='D', accion='cargar', voltear=''): # m y returns temporales
    """Aplica o elimina las cargas distribuidas y las reacciones en los nodos a todas las vigas."""
    if accion == 'cargar':
        #for m in miembro:
            if not m: return #continue
            if m.name != 'VIGA': return #continue
            if caso != 'D' and caso != 'L':
                print('no trollees! que me crasheo.')
                return #break
            #if m.losa[caso][0] == 'existe': print(f'ya estaba cargado este caso ({caso}).')
            else:
                new_index = len(m.dst_loads)
                m.losa[caso].append(new_index)
                if voltear == 'mirror':
                    q_losa = h_trib_losa(m.L, d_losas, forma, True) # [a, b, h]
                else:
                    q_losa = h_trib_losa(m.L, d_losas, forma) # [a, b, h]
                q_losa[2] *= Q_L if caso == 'L' else t_losas*m.𝛾 # [a, b, q]
                # rectificacion
                k = (2*m.L - q_losa[0] - q_losa[1])/(2*m.L)
                if struct.VERB:
                    print('L', m.L, 'a', q_losa[0], 'b', q_losa[1])
                    print('k', k)
                    print('q_losa antes', q_losa)
                q_losa = [0, 0, k*q_losa[2]]
                if struct.VERB: print('q_losa despues', q_losa)
                m.dst_loads.append((
                    '_q-losa',
                    caso,
                    tuple(q_losa)
                ))
                m.jnt_loads.append((
                    '_q-losa',
                    caso,
                    array( emp_perf_Y(m.L, *m.dst_loads[m.losa[caso][-1]][2]) )
                ))
                print(f'se ha cargado la losa (caso: {caso}) en el elemento {m.name}-i:{m.i}.')
                line()
                line()
                #m.losa[caso][0] = 'existe'
    elif accion == 'l': # limpiar... pero algun dia
        print('nada que ver aqui :v *se crashea*')
        raise
    return

In [338]:
if TEST == 3:
    if 1: # !! no sobrescribe
        q_losa(miembro[17], '2', 'D', 'cargar', 'mirror') # piso 1
        q_losa(miembro[17], '3c', 'D', 'cargar', 'mirror')
        q_losa(miembro[18], '3u', 'D')
        q_losa(miembro[18], '3u', 'D')
        q_losa(miembro[19], '2', 'D')
        q_losa(miembro[19], '3c', 'D')
        q_losa(miembro[17], '2', 'L', 'cargar', 'mirror')
        q_losa(miembro[17], '3c', 'L', 'cargar', 'mirror')
        q_losa(miembro[18], '3u', 'L')
        q_losa(miembro[18], '3u', 'L')
        q_losa(miembro[19], '2', 'L')
        q_losa(miembro[19], '3c', 'L')
        q_losa(miembro[20], '2', 'D', 'cargar', 'mirror') # piso 2
        q_losa(miembro[20], '3c', 'D', 'cargar', 'mirror')
        q_losa(miembro[21], '3u', 'D')
        q_losa(miembro[21], '3u', 'D')
        q_losa(miembro[22], '2', 'D')
        q_losa(miembro[22], '3c', 'D')
        q_losa(miembro[20], '2', 'L', 'cargar', 'mirror')
        q_losa(miembro[20], '3c', 'L', 'cargar', 'mirror')
        q_losa(miembro[21], '3u', 'L')
        q_losa(miembro[21], '3u', 'L')
        q_losa(miembro[22], '2', 'L')
        q_losa(miembro[22], '3c', 'L')
        q_losa(miembro[23], '2', 'D', 'cargar', 'mirror') # piso 3
        q_losa(miembro[23], '3c', 'D', 'cargar', 'mirror')
        q_losa(miembro[24], '3u', 'D')
        q_losa(miembro[24], '3u', 'D')
        q_losa(miembro[25], '2', 'D')
        q_losa(miembro[25], '3c', 'D')
        q_losa(miembro[23], '2', 'L', 'cargar', 'mirror')
        q_losa(miembro[23], '3c', 'L', 'cargar', 'mirror')
        q_losa(miembro[24], '3u', 'L')
        q_losa(miembro[24], '3u', 'L')
        q_losa(miembro[25], '2', 'L')
        q_losa(miembro[25], '3c', 'L')
        q_losa(miembro[26], '2', 'D', 'cargar', 'mirror') # piso 4
        q_losa(miembro[26], '3c', 'D', 'cargar', 'mirror')
        q_losa(miembro[27], '3u', 'D')
        q_losa(miembro[27], '3u', 'D')
        q_losa(miembro[28], '2', 'D')
        q_losa(miembro[28], '3c', 'D')
        q_losa(miembro[26], '2', 'L', 'cargar', 'mirror')
        q_losa(miembro[26], '3c', 'L', 'cargar', 'mirror')
        q_losa(miembro[27], '3u', 'L')
        q_losa(miembro[27], '3u', 'L')
        q_losa(miembro[28], '2', 'L')
        q_losa(miembro[28], '3c', 'L')
    [print(m.dst_loads) for m in miembro if m];
    [print(m.jnt_loads) for m in miembro if m];
elif TEST == 1:
    if 1: # !! no sobrescribe
        q_losa(miembro[3], '2', 'D')
        q_losa(miembro[3], '3c', 'D')
        q_losa(miembro[3], '2', 'L')
        q_losa(miembro[3], '3c', 'L')
    [print(m.dst_loads) for m in miembro if m];
    [print(m.jnt_loads) for m in miembro if m];

L 700.0 a 256.2177826491072 b 443.7822173508929
k 0.49999999999999994
q_losa antes [256.2177826491072, 443.7822173508929, 16.64183315065848]
q_losa despues [0, 0, 8.320916575329239]
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
Ra Ma Rb Mb 2912.3 339770.8 2912.3 -339770.8
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
---- -- - -- ----
Va: 2912.32 
Ma: 339770.76 
Vb: 2912.32 
Mb: -339770.76
se ha cargado la losa (caso: D) en el elemento VIGA-i:1.
---- -- - -- ----
---- -- - -- ----
L 700.0 a 202.0725942163691 b 350.0
k 0.6056624327025936
q_losa antes [202.0725942163691, 350.0, 13.125]
q_losa despues [0, 0, 7.949319429221541]
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
Ra Ma Rb Mb 2782.3 324597.2 2782.3 -324597.2
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
---- -- - -- ----
Va: 2782.26 
Ma: 324597.21 
Vb: 2782.26 
Mb: -324597.21
se ha cargado la losa (caso: D) en el elemento VIGA-i:1.
---- -- - -- ----
---- -- - -- ----
L 700.0 a 350.0 b 350.0
k 0.5
q_losa antes [350.0, 350.0, 13.125]
q_losa despues [0, 0, 6.5625]
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
Ra Ma R

Vb: 2296.88 
Mb: -267968.75
se ha cargado la losa (caso: D) en el elemento VIGA-i:10.
---- -- - -- ----
---- -- - -- ----
L 700.0 a 350.0 b 350.0
k 0.5
q_losa antes [350.0, 350.0, 13.125]
q_losa despues [0, 0, 6.5625]
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
Ra Ma Rb Mb 2296.9 267968.8 2296.9 -267968.8
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
---- -- - -- ----
Va: 2296.88 
Ma: 267968.75 
Vb: 2296.88 
Mb: -267968.75
se ha cargado la losa (caso: D) en el elemento VIGA-i:10.
---- -- - -- ----
---- -- - -- ----
L 700.0 a 443.7822173508929 b 256.2177826491072
k 0.49999999999999994
q_losa antes [443.7822173508929, 256.2177826491072, 16.64183315065848]
q_losa despues [0, 0, 8.320916575329239]
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
Ra Ma Rb Mb 2912.3 339770.8 2912.3 -339770.8
Ra Ma Rb Mb 0.0 0.0 0.0 -0.0
---- -- - -- ----
Va: 2912.32 
Ma: 339770.76 
Vb: 2912.32 
Mb: -339770.76
se ha cargado la losa (caso: D) en el elemento VIGA-i:11.
---- -- - -- ----
---- -- - -- ----
L 700.0 a 350.0 b 202.0725942163691
k 0.6056624327025935
q

[('_pp-bar', 'D', array([   0. , 3037.5,    0. ,    0. , 3037.5,    0. ]))]
[('_pp-bar', 'D', array([   0. , 3037.5,    0. ,    0. , 3037.5,    0. ]))]
[('_pp-bar', 'D', array([   0. , 3037.5,    0. ,    0. , 3037.5,    0. ]))]
[('_pp-bar', 'D', array([   0. , 3037.5,    0. ,    0. , 3037.5,    0. ]))]
[('_pp-bar', 'D', array([   0. , 3037.5,    0. ,    0. , 3037.5,    0. ]))]
[('_pp-bar', 'D', array([      0.,    3150.,  367500.,       0.,    3150., -367500.])), ('_q-losa', 'D', array([      0.        ,    2912.32080137,  339770.76015928,
             0.        ,    2912.32080137, -339770.76015928])), ('_q-losa', 'D', array([      0.        ,    2782.26180023,  324597.21002655,
             0.        ,    2782.26180023, -324597.21002655])), ('_q-losa', 'L', array([      0.        ,    3883.09440182,  453027.68021237,
             0.        ,    3883.09440182, -453027.68021237])), ('_q-losa', 'L', array([      0.        ,    3709.6824003 ,  432796.28003539,
             0.        ,    

In [339]:
# cargar manualmente cargas distribuidas - in the future
def load_dist(): pass

len(miembro)

29

## Analisis estatico

- [volver](#Indice)

Se genera un vector de fuerzas solicitantes, E, a partir del metodo
estatico.

In [340]:
# cargas sismicas - metodo estatico
def metodo_estatico(zona, categoria, suelo, R, h_pisos, n_pisos, d_losas, n_vanos):    
    """Calcula un vector con las fuerzas horizontales producto del sismo."""
    TABLA_I = { # Tabla 6.1
        # "CategoriaDeEdificio": I [factor de importancia],
        "I"  : 0.6,
        "II" : 1.0,
        "III": 1.2,
        "IV" : 1.2,
    }

    TABLA_A0 = { # Tabla 6.2
        # "ZonaSismica": A0/g,
        "1": 0.20,
        "2": 0.30,
        "3": 0.40,
    }

    TABLA_PARAMETROS_SUELO = { # Tabla 6.3
        # "TipoDeSuelo": (S, T0, Tp [T'], n, p),
        "A": (0.90, 0.15, 0.20, 1.00, 2.0),
        "B": (1.00, 0.30, 0.35, 1.33, 1.5),
        "C": (1.05, 0.40, 0.45, 1.40, 1.6),
        "D": (1.20, 0.75, 0.85, 1.80, 1.0),
        "E": (1.30, 1.20, 1.35, 1.80, 1.0),
        "F": (None, None, None, None, None),
    }

    TABLA_C_MAX = { # Tabla 6.4
        # "R": CoeficienteC_max,
        "2"  : 0.90,
        "3"  : 0.60,
        "4"  : 0.55,
        "5.5": 0.40,
        "6"  : 0.35,
        "7"  : 0.35,
    }

    Tn = round(0.09*h_pisos*n_pisos/(d_losas/100*n_vanos)**0.5, 4)

    A0 = TABLA_A0[zona] # aceleracion efectiva
    I = TABLA_I[categoria]
    S, T0, Tp, n, p = TABLA_PARAMETROS_SUELO[suelo]

    # coeficiente sismico
    Cmax = round(TABLA_C_MAX[str(R)]*S*A0, 8)
    Cmin = A0*S/6
    C = 2.75*S*A0/R*(Tp/Tn)**n

    if C < Cmin: C = Cmin
    elif C > Cmax: C = Cmax


    # numero de elementos por piso
    n_viga = n_marcos*n_vanos
    n_colum = n_marcos*(n_vanos + 1)
    n_vig_u = (n_marcos - 1)*(n_vanos + 1) # viga de union de marcos

    # masa por elemento
    for m in miembro:
        if not m: continue
        if m.name == 'VIGA':
            m_viga = m.𝛾*m.A*m.L
            print('viga: gamma, area, largo', m.𝛾, m.A, m.L)
            m_vig_u = m.𝛾*m.A*d_losas
            break
    for m in miembro:
        if not m: continue
        if m.name == 'COL':
            m_colum = m.𝛾*m.A*m.L
            print('col: gamma, area, largo', m.𝛾, m.A, m.L)
            break

    if struct.VERB:
        print('numero de v, v_u, c:', n_viga, n_vig_u, n_colum)
        print('masa de v, v_u, c:', m_viga, m_vig_u, m_colum)

    # masa estructura por piso
    M_st = (n_viga*m_viga) + (n_colum*m_colum) + (n_vig_u*m_vig_u)
    print(f'n y m: viga {n_viga} de {m_viga}, col {n_colum} de {m_colum}, viga_un {n_vig_u} de {m_vig_u}')
    # masa losas por piso
    M_l = m.𝛾 * b_losas*d_losas*t_losas * n_vanos*(n_marcos - 1)

    # masa sismica por nivel
    MUERTO = M_st + M_l # [kg]
    VIVO = 0.25*Q_L * b_losas*n_vanos * d_losas*(n_marcos - 1) # [kg]
    Pk = [0]
    for nivel in range(n_pisos):
        if nivel == n_pisos - 1:
            Pk.append(MUERTO - n_colum*m_colum/2 + VIVO)
            break
        Pk.append(MUERTO + VIVO)
    print('masas por nivel:', Pk, 'largo pk:', len(Pk))

    P = sum(Pk) # [kg] - peso sismico
    Q0 = C*I*P # [kg] - corte basal

    print(
        'testing vars:\n',
        f'A0: {A0}, I: {I}, S: {S}, T0: {T0}\n',
        f'Tp: {Tp}, n: {n}, p: {p}\n',
        f'Cmax: {Cmax}, Cmin: {round(Cmin, 4)}, C: {round(C, 4)}\n',
        f'Q0: {Q0}, Pk: {Pk}, P: {P}',
        f'\nperiodo fundamental: {Tn}'
    ) # borrame
    print('masas: M_st {}, M losa {}'.format(M_st, M_l))
    print('muerto y vivo:', MUERTO, VIVO)

    H = round(h_pisos*n_pisos, 4)
    Z = [0] + [round(i*h_pisos, 4) for i in range(1, n_pisos + 1)]
    A = [0]
    for k, _ in enumerate(Z):
        if k == 0: continue
        A.append(
            (1 - Z[k - 1]/H)**0.5 - (1 - Z[k]/H)**0.5
        )
        print('nivel {}, altura acumulada {}, {}%'.format(k, Z[k], round(A[k], 3))) # borrame

    P = Pk # mejor coincidencia con formulas de la nch433
    del Pk

    print('largos:', len(A), len(P))
    F = []
    AjPj = sum([i*k for i, k in zip(A, P)]) # Σ Aj*Pj
    for k, _ in enumerate(P):
        F.append(A[k]*P[k]/AjPj*Q0)
        print('AP/AP:', A[k]*P[k]/AjPj)

    print('A*P', AjPj) # borrame
    print('F por nivel:', F) # borrame

    fx = []
    for k, _ in enumerate(F):
        if k == 0: continue
        for i in range(n_vanos + 1):
            fx.append(F[k]/((n_vanos + 1)*(n_marcos)))
            fx.append(0)
            fx.append(0)
    return array(fx).T

E = metodo_estatico(zona, categoria, suelo, R, h_pisos/100, n_pisos, d_losas, n_vanos)
E

viga: gamma, area, largo 0.0025 3600 700.0
col: gamma, area, largo 0.0025 8100 300.0
numero de v, v_u, c: 12 12 16
masa de v, v_u, c: 6300.0 6300.0 6075.0
n y m: viga 12 de 6300.0, col 16 de 6075.0, viga_un 12 de 6300.0
masas por nivel: [0, 468900.0, 468900.0, 468900.0, 420300.0] largo pk: 5
testing vars:
 A0: 0.4, I: 1.0, S: 1.3, T0: 1.2
 Tp: 1.35, n: 1.8, p: 1.0
 Cmax: 0.182, Cmin: 0.0867, C: 0.182
 Q0: 332514.0, Pk: [0, 468900.0, 468900.0, 468900.0, 420300.0], P: 1827000.0 
periodo fundamental: 0.2357
masas: M_st 248400.0, M losa 165375.0
muerto y vivo: 413775.0 55125.0
nivel 1, altura acumulada 3.0, 0.134%
nivel 2, altura acumulada 6.0, 0.159%
nivel 3, altura acumulada 9.0, 0.207%
nivel 4, altura acumulada 12.0, 0.5%
largos: 5 5
AP/AP: 0.0
AP/AP: 0.14129709438928642
AP/AP: 0.1676044582459539
AP/AP: 0.21842638258743177
AP/AP: 0.4726720647773279
A*P 444600.0
F por nivel: [0.0, 46983.26204375918, 55730.82882919512, 72629.83017967729, 157170.07894736843]


array([2936.45387773,    0.        ,    0.        , 2936.45387773,
          0.        ,    0.        , 2936.45387773,    0.        ,
          0.        , 2936.45387773,    0.        ,    0.        ,
       3483.17680182,    0.        ,    0.        , 3483.17680182,
          0.        ,    0.        , 3483.17680182,    0.        ,
          0.        , 3483.17680182,    0.        ,    0.        ,
       4539.36438623,    0.        ,    0.        , 4539.36438623,
          0.        ,    0.        , 4539.36438623,    0.        ,
          0.        , 4539.36438623,    0.        ,    0.        ,
       9823.12993421,    0.        ,    0.        , 9823.12993421,
          0.        ,    0.        , 9823.12993421,    0.        ,
          0.        , 9823.12993421,    0.        ,    0.        ])

## Combinaciones de carga

- [volver](#Indice)

Se genera un vector de fuerzas, _S_, para cada combinacion de cargas.

In [341]:
# vector de fuerzas segun casos de carga
#    1.4*D
#    1.2*D + 1.6*L
#    1.2*D + 1.4*E + L
#    1.2*D - 1.4*E + L
#    0.9*D + 1.4*E
#    0.9*D - 1.4*E

def loads_gen(accion='generar'): # convertir en metodo
    """Genera las categorias y combinaciones de carga de la estructura."""
    if accion == 'generar':
        case_gen = ['D', 'L', 'E']
        struct.caso = {}
        for i in case_gen:
            struct.caso.update({ i: zeros(len(struct.K)) })
        
        combo_gen = [
            '1.4 D',
            '1.2 D + 1.6 L',
            '1.2 D + L',
            'E',
            '- E',
            '1.2 D + 1.4 E + L',
            '1.2 D - 1.4 E + L',
            '0.9 D + 1.4 E',
            '0.9 D - 1.4 E',
        ]
        struct.combo = {}
        struct.r = {}
        for m in miembro:
            if not m: continue
            if 1: m.q_combo.clear() # temp carga rectangular
            m.combo.clear()
        for i in combo_gen:
            struct.combo.update({ i: zeros(len(struct.K)) })
            struct.r.update({ i: zeros(len(struct.K)) })
            for m in miembro:
                if not m or i == 'E' or i == '- E': continue
                if struct.VERB: print(f'limpiando y generando {m.name}')
                if 1: m.q_combo.update({ i: 0 }) # temp carga rectangular
                m.combo.update({ i: zeros(len(struct.K)) })
    elif accion == 'cargar': pass
    return

loads_gen()

limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando VIGA
limpiando y generando VIGA
limpiando y generando VIGA
limpiando y generando VIGA
limpiando y generando VIGA
limpiando y generando VIGA
limpiando y generando VIGA
limpiando y generando VIGA
limpiando y generando VIGA
limpiando y generando VIGA
limpiando y generando VIGA
limpiando y generando VIGA
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL
limpiando y generando COL


In [342]:
for i, m in enumerate(miembro):
    if not m: continue
    m.case_gen()
    for F in m.jnt_loads:
        if 0: # borrame
            print(
                f'Fuente: miembro {i}',
                f'perfil: {m.name}',
                f'nombre: \'{F[0]}\'',
                f'caso: \'{F[1]}\'',
                f'vector: {F[2]}\n',
                sep='\n'
            )
        if F[1] == 'D':
            m.caso['D'] += F[2]
            struct.caso['D'] -= F[2]@m.T
        elif F[1] == 'L':
            m.caso['L'] += F[2]
            struct.caso['L'] -= F[2]@m.T
    if 1: # temp rect
        for F in m.dst_loads:
            if F[1] == 'D':
                m.q_caso['D'] += F[2][2]
            if F[1] == 'L':
                m.q_caso['L'] += F[2][2]

if TEST != 2:
    struct.caso['E'] = metodo_estatico(zona, categoria, suelo, R, h_pisos/100, n_pisos, d_losas, n_vanos)

if 0: # nani
    for m in miembro:
        if m: print(f'\nperfil \'{m.name}\'', m.caso, sep='\n')

if 1: # temp
    for m in miembro:
        if not m: continue
        # cargas rectangulares por combinacion
        m.q_combo['1.4 D'] = \
            1.4*m.q_caso['D']

        m.q_combo['1.2 D + 1.6 L'] = \
            1.2*m.q_caso['D'] + 1.6*m.q_caso['L']

        m.q_combo['1.2 D + L'] = \
            1.2*m.q_caso['D'] + m.q_caso['L']

        m.q_combo['1.2 D + 1.4 E + L'] = \
            1.2*m.q_caso['D'] + m.q_caso['L']

        m.q_combo['1.2 D - 1.4 E + L'] = \
            1.2*m.q_caso['D'] + m.q_caso['L']

        m.q_combo['0.9 D + 1.4 E'] = \
            0.9*m.q_caso['D']

        m.q_combo['0.9 D - 1.4 E'] = \
            0.9*m.q_caso['D']

for m in miembro:
    if not m: continue
    # vectores de empotramiento perfecto por combinacion
    m.combo['1.4 D'] = \
        1.4*m.caso['D']

    m.combo['1.2 D + 1.6 L'] = \
        1.2*m.caso['D'] + 1.6*m.caso['L']

    m.combo['1.2 D + L'] = \
        1.2*m.caso['D'] + m.caso['L']

    m.combo['1.2 D + 1.4 E + L'] = \
        1.2*m.caso['D'] + m.caso['L']

    m.combo['1.2 D - 1.4 E + L'] = \
        1.2*m.caso['D'] + m.caso['L']

    m.combo['0.9 D + 1.4 E'] = \
        0.9*m.caso['D']

    m.combo['0.9 D - 1.4 E'] = \
        0.9*m.caso['D']

# vectores de fuerza por combinacion
struct.combo['1.4 D'] = \
    1.4*struct.caso['D']

struct.combo['1.2 D + 1.6 L'] = \
    1.2*struct.caso['D'] + 1.6*struct.caso['L']

struct.combo['1.2 D + L'] = \
    1.2*struct.caso['D'] + struct.caso['L']

struct.combo['E'] = \
    struct.caso['E']

struct.combo['- E'] = \
    -1*struct.caso['E']

struct.combo['1.2 D + 1.4 E + L'] = \
    1.2*struct.caso['D'] + 1.4*struct.caso['E'] + struct.caso['L']

struct.combo['1.2 D - 1.4 E + L'] = \
    1.2*struct.caso['D'] - 1.4*struct.caso['E'] + struct.caso['L']

struct.combo['0.9 D + 1.4 E'] = \
    0.9*struct.caso['D'] + 1.4*struct.caso['E']

struct.combo['0.9 D - 1.4 E'] = \
    0.9*struct.caso['D'] - 1.4*struct.caso['E']

vars(struct)

viga: gamma, area, largo 0.0025 3600 700.0
col: gamma, area, largo 0.0025 8100 300.0
numero de v, v_u, c: 12 12 16
masa de v, v_u, c: 6300.0 6300.0 6075.0
n y m: viga 12 de 6300.0, col 16 de 6075.0, viga_un 12 de 6300.0
masas por nivel: [0, 468900.0, 468900.0, 468900.0, 420300.0] largo pk: 5
testing vars:
 A0: 0.4, I: 1.0, S: 1.3, T0: 1.2
 Tp: 1.35, n: 1.8, p: 1.0
 Cmax: 0.182, Cmin: 0.0867, C: 0.182
 Q0: 332514.0, Pk: [0, 468900.0, 468900.0, 468900.0, 420300.0], P: 1827000.0 
periodo fundamental: 0.2357
masas: M_st 248400.0, M losa 165375.0
muerto y vivo: 413775.0 55125.0
nivel 1, altura acumulada 3.0, 0.134%
nivel 2, altura acumulada 6.0, 0.159%
nivel 3, altura acumulada 9.0, 0.207%
nivel 4, altura acumulada 12.0, 0.5%
largos: 5 5
AP/AP: 0.0
AP/AP: 0.14129709438928642
AP/AP: 0.1676044582459539
AP/AP: 0.21842638258743177
AP/AP: 0.4726720647773279
A*P 444600.0
F por nivel: [0.0, 46983.26204375918, 55730.82882919512, 72629.83017967729, 157170.07894736843]


{'grilla': [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]],
 'dimXY': (4, 4),
 'dim': {'x': 4, 'y': 4},
 'nodos': 16,
 'max_barras': 28,
 'K': array([[ 2.74127032e+06,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  1.48219297e+07,  8.15434799e+06, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  8.15434799e+06,  4.37616676e+10, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          2.07533190e+06,  0.00000000e+00,  9.98907629e+07],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  7.42261391e+06, -8.15434799e+06],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          9.98907629e+07, -8.15434799e+06,  2.37835150e+10]]),
 'r': {'1.4 D': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

## Resultados: esfuerzos nodales

- [volver](#Indice)

Luego de obtenidos los vectores de fuerzas para cada combinacion, se
resuelve por cada uno la estructura y se calculan los esfuerzos en
los extremos de cada elemento para cada combinacion a partir de los
vectores de desplazamientos nodales.

El siguiente texto no envejecio bien:

```
Se busca la combinacion de carga que controla el diseño, luego se
busca en la estructura la viga y la columna mas desfavorable. En el
caso de la viga, se calcula el momento a $\frac{b}{2}$, donde $b$ es
el ancho de la columna que sustenta la viga.
```

In [343]:
line()
struct.r = {}
for COMBO in struct.combo.keys():
    struct.r.update({ COMBO: inv(struct.K)@struct.combo[COMBO] }) # desplazamientos

for m in miembro:
    if not m: continue
    # inicializacion
    m.r = {}
    m.ε = {}
    m.σ = {}
    m.v = {}
    m.sigma = {}
    for COMBO in m.combo.keys():
        # definicion
        m.r.update({ COMBO: m.T@struct.r[COMBO] })
        m.ε.update({ COMBO: m.a@m.r[COMBO] })
        m.σ.update({ COMBO: m.k@m.ε[COMBO] })
        m.v.update({ COMBO: (m.σ[COMBO][0] + m.σ[COMBO][1])/m.L })
        m.sigma.update({ COMBO: m.a.T@m.σ[COMBO] })
for i, m in enumerate(miembro):
    if m and struct.VERB: print(f'miembro {i}, vector de esfuerzos:\n', m.sigma)
#vars(miembro[1])

for m in miembro:
    if not m: continue
    m.resumen = {}
    for COMBO in m.sigma.keys():
        m.resumen.update({ COMBO: mround(m.sigma[COMBO] + m.combo[COMBO], 2) })

---- -- - -- ----
miembro 1, vector de esfuerzos:
 {'1.4 D': array([   3518.34013013,   79058.06911002, -390163.24868724,
         -3518.34013013,  -79058.06911002, -665338.79035271]), '1.2 D + 1.6 L': array([    6456.67645178,   116125.74639248,  -715208.64935225,
          -6456.67645178,  -116125.74639248, -1221794.28618181]), '1.2 D + L': array([   5166.31782419,   97990.11370924, -572415.02149462,
         -5166.31782419,  -97990.11370924, -977480.32576272]), '1.2 D + 1.4 E + L': array([ -20093.5437428 ,   61650.97202329, 6540056.27950225,
         20093.5437428 ,  -61650.97202329, -511993.15666246]), '1.2 D - 1.4 E + L': array([   30426.17939118,   134329.25539518, -7684886.3224915 ,
         -30426.17939118,  -134329.25539518, -1442967.49486298]), '0.9 D + 1.4 E': array([ -22998.07148333,   14483.90274192, 6861652.06969794,
         22998.07148333,  -14483.90274192,   37769.37530209]), '0.9 D - 1.4 E': array([   27521.65165065,    87162.18611382, -7363290.53229581,
         -275

       -5.68648973e+03,  1.96551639e+06])}
miembro 27, vector de esfuerzos:
 {'1.4 D': array([ 5.16364444e+03,  1.05249143e-12,  4.76691951e+03, -5.16364444e+03,
       -1.05209638e-12, -4.76691951e+03]), '1.2 D + 1.6 L': array([ 9.05862440e+03,  1.32516220e-12,  1.69757515e+04, -9.05862440e+03,
       -1.32378245e-12, -1.69757515e+04]), '1.2 D + L': array([ 7.32138311e+03,  1.21502808e-12,  1.21420688e+04, -7.32138311e+03,
       -1.21542885e-12, -1.21420688e+04]), '1.2 D + 1.4 E + L': array([    7321.3831067 ,    -5515.01675662, -1918113.79597949,
          -7321.3831067 ,     5515.01675662, -1942397.93365459]), '1.2 D - 1.4 E + L': array([   7321.3831067 ,    5515.01675662, 1942397.93365459,
         -7321.3831067 ,   -5515.01675662, 1918113.79597949]), '0.9 D + 1.4 E': array([    3319.48571278,    -5515.01675662, -1927191.41656365,
          -3319.48571278,     5515.01675662, -1933320.31307043]), '0.9 D - 1.4 E': array([   3319.48571278,    5515.01675662, 1933320.31307043,
        

## Resultados: Resumen

- [volver](#Indice)

A continuacion se presenta una tabla que resume las cargas y momentos
puntuales que actuan sobre cada elemento. Las cargas que se presentan
corresponden a cada extremo, $a$ y $b$, de los diagramas de esfuerzos
internos. Tabien se presenta una tabla que resume los desplazamientos
de los nodos por cada combinacion ($\delta_x$, $\delta_y$, Giro).
Finalmente, se presenta una tabla que muestra las deformaciones por
nivel y el drift (‰) de cada uno para cada combinacion.

In [344]:
def reporte(resumen='esfuerzos'):
    """Genera los reportes para esfuerzos en los extremos de cada\
    elemnto, desplazamientos nodales, desplazamientos por nivel y\
    drift por nivel para cada combinacion y les da formato en un\
    archivo por cada tabla."""
    if resumen != 'generar':
        tabla = struct.annainfo[resumen]
    if resumen == 'generar':
        struct.clean()
        tabla = struct.annainfo['esfuerzos']
        for k, m in enumerate(miembro):
            if not m: continue
            for COMBO in m.resumen:
                if m.name == 'VIGA':
                    tabla['body'].append([
                        m.name,
                        k,
                        COMBO,
                        'i',
                        m.resumen[COMBO][0],
                        m.resumen[COMBO][1],
                        m.resumen[COMBO][2]/100,
                    ])
                    tabla['body'].append([
                        'j',
                        m.resumen[COMBO][3]*-1,
                        m.resumen[COMBO][4]*-1,
                        m.resumen[COMBO][5]/-100,
                    ])
                elif m.name == 'COL':
                    tabla['body'].append([
                        m.name,
                        k,
                        COMBO,
                        'i',
                        m.resumen[COMBO][1],
                        m.resumen[COMBO][0]*-1,
                        m.resumen[COMBO][2]/100,
                    ])
                    tabla['body'].append([
                        'j',
                        m.resumen[COMBO][4]*-1,
                        m.resumen[COMBO][3],
                        m.resumen[COMBO][5]/-100,
                    ])
        tabla = struct.annainfo['delta-nodos']
        for combo in struct.r:
            for nodo in range(struct.nodos):
                tabla['body'].append([
                    combo,
                    nodo + 1,
                    struct.r[combo][3*nodo]*10,
                    struct.r[combo][3*nodo + 1]*10,
                    struct.r[combo][3*nodo + 2],
                ])
        tabla = struct.annainfo['drifts']
        for combo in ['E', '- E']:
            struct.drift.update({ combo: [] })
            δmax1, δmin1, δmax2, δmin2 = 0, 0, 0, 0
            nivel = 0
            deltaX = []
            for i in range(len(struct.r[combo])):
                if 3*i >= len(struct.r[combo]): break
                deltaX.append(struct.r[combo][3*i])
            for i in range(len(deltaX)):
                if struct.dim['x']*i >= len(deltaX): break
                nivel += 1
                for j in deltaX[struct.dim['x']*i:struct.dim['x']*(i + 1)]:
                    if deltaX[struct.dim['x']*i] == j:
                        δmin2 = j
                    if abs(j) > abs(δmax2):
                        δmax2 = j
                    elif abs(j) < abs(δmin2):
                        δmin2 = j
                drift = round((δmax2 - δmin1)/h_pisos*1000, 3) # [‰]
                δmax1, δmin1 = δmax2, δmin2
                tabla['body'].append([
                    combo,
                    nivel,
                    10*δmax2,
                    10*δmin2,
                    drift,
                ])
                struct.drift[combo].append(drift)
    elif resumen == 'esfuerzos':
        print(' '*30 + '{:>19}{:>13}{:>13}'.format(*tabla['head'][1]))
        print('fila  {:^6} {:^19} {:^4}{:>12}{:>13}{:>13}'.format(*tabla['head'][0]))
        i = 1
        print('='*75)
        for fila in tabla['body']:
            #if i == 12: break
            if i%2 != 0:
                print(f'{i:<6}' + '{:<4}{:>2} {:^19} ({}) {:>12} {:>12} {:>12.2f}'.format(*fila))
            else:
                print(f'{i:<6}' + ' '*27 + '({}) {:>12} {:>12} {:>12.2f}'.format(*fila))
                print('-'*75)
            i += 1
    elif resumen == 'delta-nodos':
        print(' '*22 + '{:>21}{:>12}{:>14}'.format(*tabla['head'][1]))
        print('fila  {:^19} {:4} {:>12} {:>11} {:>13}'.format(*tabla['head'][0]))
        i = 1
        print('='*69)
        for fila in tabla['body']:
            #if i == 12: break
            print(f'{i:<6}' + '{:^19} ({:02}) {:>12.4f} {:>11.4f} {:>13.3e}'.format(*fila))
            print('-'*69)
            i += 1
    elif resumen == 'drifts':
        print(' '*22 + '{:>21}{:>12}{:>11}'.format(*tabla['head'][1]))
        print('fila  {:^19} {:4} {:>11} {:>11} {:>10}'.format(*tabla['head'][0]))
        i = 1
        print('='*66)
        for fila in tabla['body']:
            #if i == 12: break
            print(f'{i:<6}' + '{:^19} ({:02}) {:>12.4f} {:>11.4f} {:>10.3f}'.format(*fila))
            print('-'*66)
            i += 1
    return

reporte('generar')
#reporte('esfuerzos')
#reporte('delta-nodos')
reporte('drifts')

                                       [mm]        [mm]        [‰]
fila         combo        nivel        δmax        δmin      drift
1              E          (01)       0.9953      0.9839      0.332
------------------------------------------------------------------
2              E          (02)       2.6835      2.6818      0.567
------------------------------------------------------------------
3              E          (03)       4.2595      4.2574      0.526
------------------------------------------------------------------
4              E          (04)       5.4373      5.4103      0.393
------------------------------------------------------------------
5             - E         (01)      -0.9953     -0.9839     -0.332
------------------------------------------------------------------
6             - E         (02)      -2.6835     -2.6818     -0.567
------------------------------------------------------------------
7             - E         (03)      -4.2595     -4.2574     -0

In [345]:
ψ = 0
d = 0
for i in struct.drift:
    for j in struct.drift[i]:
        if abs(j) > d:
            d = abs(j)
            r = abs(j)/(2/1.2)
ψ = r**0.25

p(f'drift: {d}, r: {r}, ψ: {ψ}')

drift: 0.567, r: 0.34019999999999995, ψ: 0.7637189685065491


#### factor

[hack](#hack)

In [346]:
p(f'drift maximo: {d}\n',
  f'ψ: {ψ}', sep='')
line()

if 1:
    perfiles_i = [(m.b, m.h) for m in miembro if m]
    perfiles_f = [(round(m.b*ψ), round(m.h*ψ)) for m in miembro if m]
    if perfiles_i == perfiles_f: p('verifica para el drift!')
    else: p('se necesita corregir los perfiles...')
    line()
    for m in miembro:
        if not m: continue
        p(m.b, m.h, m.name)
    line()
    for m in miembro:
        if not m: continue
        p(
            (round(m.b*ψ), round(m.h*ψ)), ',',
            sep='', end=''
        )
    p()
    line()

drift maximo: 0.567
ψ: 0.7637189685065491
---- -- - -- ----
se necesita corregir los perfiles...
---- -- - -- ----
90 90 COL
90 90 COL
90 90 COL
90 90 COL
90 90 COL
90 90 COL
90 90 COL
90 90 COL
90 90 COL
90 90 COL
90 90 COL
90 90 COL
90 90 COL
90 90 COL
90 90 COL
90 90 COL
40 90 VIGA
40 90 VIGA
40 90 VIGA
40 90 VIGA
40 90 VIGA
40 90 VIGA
40 90 VIGA
40 90 VIGA
40 90 VIGA
40 90 VIGA
40 90 VIGA
40 90 VIGA
---- -- - -- ----
(69, 69),(69, 69),(69, 69),(69, 69),(69, 69),(69, 69),(69, 69),(69, 69),(69, 69),(69, 69),(69, 69),(69, 69),(69, 69),(69, 69),(69, 69),(69, 69),(31, 69),(31, 69),(31, 69),(31, 69),(31, 69),(31, 69),(31, 69),(31, 69),(31, 69),(31, 69),(31, 69),(31, 69),
---- -- - -- ----


In [347]:
def porto():
    _perfiles = []
    for m in miembro:
        if not m: continue
        _perfiles.append((m.b, m.h))
    dick = struct.annainfo['esfuerzos'].copy()
    dick.update({ 'perfiles': _perfiles })
    dick.update({ 'drift': d })
    with open('_pera.py', 'w') as _pera:
        _pera.write('tabla = ' + str(dick))

porto()
del porto

[hack](#hack)

la funcion `porto()` exporta los esfuerzos y perfiles de la ultima
vuelta.

In [348]:
if 0:
    import opti
    p(opti.jandro())
    del opti

## Resultados: Diagramas

- [volver](#Indice)

Las cargas aceptadas por el programa son casos genericos de cargas
trapezoidales. Las funciones se programan con esto en mente para
posteriormente obtener los respectivos diagramas de esfuerzo axial,
cortante y de momento.

In [349]:
def N(x, L, a, b, q, Na, Va, Ma):
    """Calcula el diagrama de esfuerzo axial de una viga."""
    if 0 <= x <= L:
        return Na
    else:
        return 'out of bar length!'

def V(x, L, a, b, q, Na, Va, Ma): # sera metodo
    """Calcula el diagrama de esfuerzo cortante de una viga."""
    def V1(x): return Va - q*x**2/(2*a)
    def V2(x): return V1(a) - q*(x - a)
    def V3(x): return V2(L - b) - q*(x - L + b)/2*(1 + (L - x)/b)
    
    if 0 <= x < a: return V1(x)
    elif a <= x < L - b: return V2(x)
    elif L - b <= x <= L: return V3(x)
    
    return 'out of bar length!'

def M(x, L, a, b, q, Na, Va, Ma):
    """Calcula el diagrama de momento de una viga."""
    def M1(x): return -Ma + Va*x - q*x**3/(6*a)
    def M2(x): return -Ma + Va*x - q*(a*x/2 - a**2/3) - q*(x - a)**2/2
    def M3(x): return -Ma + Va*x - q*(a*x/2 - a**2/3) - q*(L - b - a)*(x - a - (L - b - a)/2) - q*(x - L + b)**2/3*(1 + (L - x)/(2*b))
    
    if 0 <= x < a: return M1(x)
    elif a <= x < L - b: return M2(x)
    elif L - b <= x <= L: return M3(x)

    return 'out of bar length!'

## Resultados: Graficos

- [volver](#Indice)

Se exponen los diagramas de esfuerzo interno de la viga y columna mas
desfavorables y la combinacion de cargas que controla al calculo.

In [350]:
def grafVV(barras, barra, combo, ij=False):
    """Grafica los diagramas de esfuerzo interno de cada elemento."""

    # q      ( a, b, q )
    q_rect = (
        0.00000001,
        0.00000001,
        barras[barra].q_combo[combo],
    )
    Na = barras[barra].resumen[combo][0]
    Va = barras[barra].resumen[combo][1]
    Ma = barras[barra].resumen[combo][2]
    L = int(barras[barra].L)
    x, y = [], {'N':[],'V':[],'M':[]}
    ijab = {'N':[],'V':[],'M':[]}

    if barras[barra].name == 'VIGA' and ij:
        for m in barras:
            if not m: continue
            if m.j == barras[barra].i and m.name == 'COL':
                L1 = m.b/2
                L2 =  L - m.b/2
        ijab['N'].append( N(L1, L, *q_rect, Na, Va, Ma) )
        ijab['V'].append( V(L1, L, *q_rect, Na, Va, Ma) )
        ijab['M'].append( M(L1, L, *q_rect, Na, Va, Ma)/100 )
        ijab['N'].append( N(L2, L, *q_rect, Na, Va, Ma) )
        ijab['V'].append( V(L2, L, *q_rect, Na, Va, Ma) )
        ijab['M'].append( M(L2, L, *q_rect, Na, Va, Ma)/100 )
        print(
            'Extremo i:'
            f'\nN({L1}):', round( ijab['N'][0], 2 ), '[kg]'
            f'\nV({L1}):', round( ijab['V'][0], 2 ), '[kg]'
            f'\nM({L1}):', round( ijab['M'][0], 2 ), '[kg-m]'
        )
        print(
            'Extremo j:'
            f'\nN({L2}):', round( ijab['N'][1], 2 ), '[kg]'
            f'\nV({L2}):', round( ijab['V'][1], 2 ), '[kg]'
            f'\nM({L2}):', round( ijab['M'][1], 2 ), '[kg-m]'
        )

    for _x in range(L + 1):
        x.append(_x)
        y['N'].append(N(_x, L, *q_rect, Na, Va, Ma))
        y['V'].append(V(_x, L, *q_rect, Na, Va, Ma))
    if barras[barra].name == 'VIGA':
        for _x in range(L + 1):
            y['M'].append(M(_x, L, *q_rect, Na, Va, Ma))
    elif barras[barra].name == 'COL':
        for _x in range(L + 1):
            y['M'].append(M(_x, L, *q_rect[0:2], 0, Va, Na*-1, Ma))
    palo = [0 for i in x]
    x, palo = array(x), array(palo)
    if barras[barra].name == 'VIGA':
        y['N'] = array(y['N'])
        y['V'] = array(y['V'])
        y['M'] = array(y['M'])/100
    elif barras[barra].name == 'COL':
        y['N'], y['V'] = array(y['V']), array(y['N'])*-1
        y['M'] = array(y['M'])/100

    figura, ejes = pete.subplots(3, 1, figsize=(8.0,11.0))
    pete.rcParams.update({'font.size': 8.7})
    pete.subplots_adjust(hspace=0.4)
    ejes[0].set_title(f'Miembro {barra}: {barras[barra].name}', y=1.1, fontsize=19)
    for k, nvm in enumerate(['N', 'V', 'M']):
        ejes[k].grid()
        ejes[k].plot(x, y[nvm], 'k--', label=f'${nvm}(x)$')
        if barras[barra].name == 'VIGA':
            ejes[k].plot(L1, ijab[nvm][0], 'ro')
            ejes[k].plot(L2, ijab[nvm][1], 'ro')
            if 0 < min(y[nvm]) <= max(y[nvm]):
                ejes[k].plot([L1, L1], [0, max(y[nvm])], 'r', linewidth=1)
                ejes[k].plot([L2, L2], [0, max(y[nvm])], 'r', linewidth=1)
            elif min(y[nvm]) <= max(y[nvm]) < 0:
                ejes[k].plot([L1, L1], [min(y[nvm]), 0], 'r', linewidth=1)
                ejes[k].plot([L2, L2], [min(y[nvm]), 0], 'r', linewidth=1)
            else:
                ejes[k].plot([L1, L1], [min(y[nvm]), max(y[nvm])], 'r', linewidth=1)
                ejes[k].plot([L2, L2], [min(y[nvm]), max(y[nvm])], 'r', linewidth=1)
        ejes[k].plot(x, palo, 'k-')
        ejes[k].fill_between(x, y[nvm], palo, where=(y[nvm] > palo), facecolor='b', alpha=0.33)
        ejes[k].fill_between(x, y[nvm], palo, where=(y[nvm] < palo), facecolor='g', alpha=0.33)
        ejes[k].legend(loc='best', fontsize=15);
        ejes[k].set_xlabel(f'$L(x)$ [cm]', fontsize=14)
        if nvm == 'M':
            ejes[k].set_ylabel(f'${nvm}(x)$ [kg-m]', fontsize=14)
            ejes[k].invert_yaxis()
        else:
            ejes[k].set_ylabel(f'${nvm}(x)$ [kg]', fontsize=14)
    #figura.show()
    figura.savefig(f'_miembro{barra}.svg')
    return

if 0:
    if TEST == 3:
        grafVV(miembro, 18, '1.2 D + 1.4 E + L', 1)
    elif TEST == 1:
        grafVV(miembro, 3, '0.9 D - 1.4 E', 1)

## Esquema de la estructura

- [volver](#Indice)

In [351]:
#struct.dibujar()

## Flujo de la aplicacion

- [volver](#Indice)
- [hack](#hack)

A continuacion, se resume el cuerpo de la aplicacion, donde se
escribe de forma declarativa el flujo del programa.

In [352]:
#fMaxViga = [CorteA, MomentoA, CorteB, MomentoB]
#fMaxCol = [AxialA, CorteA, MomentoA, AxialB, CorteB, MomentoB]
#perfiles_0 = (anchoV, altoV, anchoC, altoC)
#perfiles_1 = (anchoV, altoV, anchoC, altoC)
#import lo_del_ale as optijandro

def pAPP():
    leerInput()
    armarEstructura()
    optimus = False
    while not optimus:
        cargasBasicas()
        cargarCombos()
        perfiles_0, esfuerzos = anna()
        perfiles_1, salida = optijandro(perfiles_0, esfuerzosV)
        perfiles_1, salida = optijandro(perfiles_0, esfuerzosC)
        if perfiles_0 == perfiles_1:
            optimus = True
        else:
            perfiles_0 = perfiles_1
            perfiles_1 = (None,)*4
            borrarFuerzas()
    reporte(salida)
    return 0

#ned

if MERYTEST:
    print(
        f'Current memory: {tracemalloc.get_traced_memory()[0]/10**6:.2f} [MB]\n',
        f'Peak          : {tracemalloc.get_traced_memory()[1]/10**6:.2f} [MB]',
        sep=''
    )
    #tracemalloc.stop()
varsC(dir())
#vars(struct)

[(1, 'COMBO'),
 (2, 'E'),
 (3, 'EP_rec'),
 (4, 'EP_tri'),
 (5, 'F'),
 (6, 'Grilla'),
 (7, 'M'),
 (8, 'MERYTEST'),
 (9, 'N'),
 (10, 'Q_L'),
 (11, 'R'),
 (12, 'TEST'),
 (13, 'V'),
 (14, 'armar_estructura'),
 (15, 'array'),
 (16, 'b_COL'),
 (17, 'b_VIG'),
 (18, 'b_losas'),
 (19, 'barra'),
 (20, 'categoria'),
 (21, 'd'),
 (22, 'd_losas'),
 (23, 'emp_perf_Y'),
 (24, 'fc'),
 (25, 'grafVV'),
 (26, 'h_COL'),
 (27, 'h_VIG'),
 (28, 'h_pisos'),
 (29, 'h_trib_losa'),
 (30, 'i'),
 (31, 'inv'),
 (32, 'j'),
 (33, 'lectura'),
 (34, 'line'),
 (35, 'load_dist'),
 (36, 'loads_gen'),
 (37, 'm'),
 (38, 'metodo_estatico'),
 (39, 'miembro'),
 (40, 'mround'),
 (41, 'n_marcos'),
 (42, 'n_pisos'),
 (43, 'n_vanos'),
 (44, 'p'),
 (45, 'pAPP'),
 (46, 'perfiles_f'),
 (47, 'perfiles_i'),
 (48, 'period'),
 (49, 'pete'),
 (50, 'pp_barras'),
 (51, 'q_losa'),
 (52, 'r'),
 (53, 'reporte'),
 (54, 'sin'),
 (55, 'solve'),
 (56, 'struct'),
 (57, 'suelo'),
 (58, 't_losas'),
 (59, 'varsC'),
 (60, 'zeros'),
 (61, 'zona'),
 (62,

Resumen

1. Introduccion
1. Objetivos
1. Alcances y Limitaciones
1. Metodologia
1. Estado del Arte
1. _licencias_
1. Analisis Estructural
1. Criterios de Diseño
1. Resultados
1. Conclusiones
- Referencias
- Bibliografia

A) Anexo A - Repositorio\
B) Anexo B - Ejercicio Resuelto\
C) Anexo C - Codigo Fuente